# Introduction to Pandas (reddit)


Full dataset at:

https://www.kaggle.com/colemaclean/subreddit-interactions

### Note about the dataset: each line represents ONE COMMENT

## Why Pandas

1. Very familiar interface / data model (Table / Excel)
2. Query data very interactively (once the data is loaded in your notebook, instant answer)
3. Complete freedom in creating custom code

GOOD TO KNOW
- Pandas is a massive library. Even after this class, there'll be a lot of things you won't know from pandas. Use the doc !


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/niparis/ga-intermediate-python-ds/master/datasets/reddit_100k.csv')

### Having a look at the data

head is a `method` of DataFrame objects. It is called by using the brackets.

head is equivalent to the SQL expression "LIMIT 5"

In [2]:
df.head()

,username,subreddit,utc
0,kabanossi,photoshopbattles,1.482748e+09
1,kabanossi,GetMotivated,1.482748e+09
2,kabanossi,vmware,1.482748e+09
3,kabanossi,carporn,1.482748e+09
4,kabanossi,DIY,1.482747e+09


We can pass a `parameter` to the `method` head. 

That's how we can control how many lines of the DataFrame `head` will display

In [20]:
# example with the parameter 2
df.head(2)

,username,subreddit,utc
0,kabanossi,photoshopbattles,1.482748e+09
1,kabanossi,GetMotivated,1.482748e+09


### Let's start we some simple data cleaning

The utc timestamp is a EPOCH timestamp in seconds

Fortunately pandas has a built-in function to transform it to a more usable *datetime* object: `pd.to_datetime`

In [5]:
# cleaning the utc timestamp: a typical "data cleaning" operation
# the result from the expression on the right of the = is *assigned* to the left expression
# df['utc'] means "selecting the column utc from the dataframe df"
df['utc'] = pd.to_datetime(df['utc'],unit='s')

In [22]:
df.head()

,username,subreddit,utc
0,kabanossi,photoshopbattles,2016-12-26 10:24:59
1,kabanossi,GetMotivated,2016-12-26 10:23:14
2,kabanossi,vmware,2016-12-26 10:21:16
3,kabanossi,carporn,2016-12-26 10:20:18
4,kabanossi,DIY,2016-12-26 10:17:59


### Let's create a new column containing only the date

We might need to have access to the date without the time information. 

It will let us aggregate the reddit data by day, to create for instance a daily timeline.

For this, we're going to use the `apply` DataFrame method. This is a very handy function that lets us modify a column in any way we want. To use it, first need to create a function that will be called on each element (or cell) of the column we want to transform.

In our case, we want to transform a `datetime` object into a `date` object, so we can write the following function:

In [3]:
def transform_to_date(elem: dt.datetime):
    # extract the date from a datetime. 
    # date is a method of the datetime object, that's why it has to be called with `()`
    return elem.date()

In [29]:
def transform_to_date_wrong(elem: dt.datetime):
    # this does no work, it just create an uncalled function
    return elem.date

Then we can apply the `transform_to_date` function to the utc column, and assign the result to a new column called `date`

In [6]:
df['date'] = df['utc'].apply(transform_to_date)
df.head(2)

,username,subreddit,utc,date
0,kabanossi,photoshopbattles,2016-12-26 10:24:59,2016-12-26
1,kabanossi,GetMotivated,2016-12-26 10:23:14,2016-12-26


Another example: week of year.
    
We create another function, apply it to the same `utc` column, and assign the result to a new column called `week_of_year`

In [16]:
def get_week_of_year(elem: dt.date):
    # weekofyear is an `attribute` from the pandas timestamp object (which is why it doesnt have to be called with `()`)
    return elem.weekofyear

df['week_of_year'] = df['utc'].apply(get_week_of_year)

## Group By

Pandas `groupby` is a powerful to perform aggregations on pandas DataFrames. It works very similarly to the SQL `GROUP BY` clause, with the added benefit of being able to define any custom aggregation function in python.

Let's start with a very simple aggregation: finding the number of comments per user.

Here's the logic of the groupby:
1. The argument of the `groupby` function (here: 'username') will be reduced to unique elements

```
df.groupby('username')
```
2. Then we select on which column we want to perform our aggregation. In our case, we want a count of comments, so any column will do:

```
df.groupby('username')['subreddit']
```

3. Finally, we select the aggregation method - in our case, a count:

```
df.groupby('username')['subreddit'].count()
```

In english, this reads:

*For each username, let's count the number of rows.*

In [31]:
# a typical groupby operation. Here we're adding sort_values and head methods to display the results nicely
df.groupby('username')['subreddit'].count().sort_values(ascending=False).head(10)


username
Spyrothedragon9972    1000
PumpTrump             1000
SidDriver             1000
bruux                 1000
Steel_Wool_Sponge     1000
SweelFor              1000
TalosThoren           1000
jackcalx              1000
TeslaAddict           1000
Tethalamin            1000
Name: subreddit, dtype: int64

#### Unique subreddits by username

We can reuse the same syntax, just changing the aggregation method

In [33]:
# How many unique subreddits per user
# in english: For each username, how many unique records can we find in the "subreddit" column
df.groupby('username')['subreddit'].nunique().sort_values(ascending=False).head(10)


username
xkcd_transcriber    496
Exuberant_Murica    188
kabanossi           187
Hcfielden           182
bigbuddha999        164
ZoomJet             154
fauxhb              154
bob-leblaw          142
gm4                 139
not_for_porns       127
Name: subreddit, dtype: int64

## Exercise 1

1. Extract the hour of day (ie: 10 for a datetime of 2016-12-26 10:24:59) and assign it to a new column ("hour_of_day")
2. Find the top 10 subreddits with the most users
3. Find which hour of the day is one average, generating more comments

In [122]:
## exercise code goes here





##

#### A more powerful syntax

pandas practitioners like to use the `.agg` method - it provides several benefits compared to the default syntax:
- Perform several aggregations on the same column
- Define the name of the results column
- Use standard python code rather than specific pandas functions


In [46]:
# Unique subreddits by username - as seen above
v0 = df.groupby('username')['subreddit'].nunique()
v0.head()

username
-goodguygeorge    106
01is               44
0mn17h3047         25
414D59             17
AbulaShabula       33
Name: subreddit, dtype: int64

#### The .agg function:

Using the `.agg` function: the argument of `.agg` is a _dictionary_ where the *keys* are the columns to aggregate on, 
and the values are the functions.


In [47]:
# In the v1, the value is a *string*, representing on of pandas default aggregation functions (here, the same nunique as
# we had been using above)
v1 = df.groupby('username').agg({'subreddit': 'nunique'})

# In v2, we're using a custom function (here a lambda function). Custom functions let us use any standard python code to 
# design our aggregation functions
# Here we reduce the aggregation to a set (which keep only unique items), and then we take the lenght of that set - which 
# is going to be the number of unique items
v2 = df.groupby('username').agg({'subreddit': lambda x: len(set(x))})

# v1 and v2 give the same results
(v1 == v2).head(10)

,subreddit
username,
-goodguygeorge,True
01is,True
0mn17h3047,True
414D59,True
AbulaShabula,True
Adjace-esque,True
Attentive_cactus,True
BasedOnAir,True
BrandEnlightened,True


### For each user, let's compute the number of comments per subreddit

Here are the steps to be able to compute this:
1. Get the comments and unique subreddits per username
2. Compute the ratio
3. Format results

First step: let's get the comments and unique subreddit in one operation


In [36]:
# The syntax to apply several aggregation on the same column is slightly different:
# we still pass a dictionnary to the `.agg` function, the key is still the column to aggregate on
# but the value becomes a *list* of tuples
# each tuple starting with the name of the result column, followed by the aggregation function
df2 = df.groupby('username').agg(
    {'subreddit': [
                    ('comments', lambda x: len(x)),
                    ('unique_subs', lambda x: len(set(x)))
                ]
    },    
)
df2.head()

subreddit            
                comments unique_subs
username                            
-goodguygeorge       978         106
01is                 353          44
0mn17h3047           211          25
414D59               117          17
AbulaShabula         363          33

In [37]:
# We get rid of the MultiIndex, it's making our life more complicated
# requires pandas 0.24 or later
df3 = df2.droplevel(axis='columns', level=0)

In [38]:
df3['comments_per_sub'] = df3['comments'] / df3['unique_subs']
df4 = df3.sort_values(by='comments_per_sub', ascending=False)
df4.head()

,comments,unique_subs,comments_per_sub
username,,,
Esstand,1000,8,125.000000
refugefirstmate,1000,12,83.333333
CyberHarassmentPanda,1000,14,71.428571
PumpTrump,1000,15,66.666667
decipedetimes10,1000,17,58.823529


### Filtering 

Filtering is quite straightforward. It uses the .loc, where the general syntax is:

```
df.loc[<column-selection>  <operator>  <value>]
```
1. `column-selection` is typically one clolumn from a database - ex: df['col']
2. `operator` is a standard python operator ( =, >, <, >=, != ...)
3. `value` can be a variable, a value, or another column


In [67]:
# A simple example, where we select all users who've made strictly less than 1000 comments
df4.loc[df4['comments'] < 1000].head()

,comments,unique_subs,comments_per_sub
username,,,
mr_fn_wonderful,999,21,47.571429
The_seph_i_am,986,32,30.812500
laserbong,804,29,27.724138
daze6,155,6,25.833333
Drums_And_Dreams,203,8,25.375000


### Using filtering to find active redditors

Let's use filtering to identify active redditors, which we'll define as users:
1. Not having exactly 1000 comments (this number looks artificial, let's remove it as a potential outlier)
2. Having either :
- Commented in at least 10 subreddits
- At least 3 comments per subreddit


In [71]:
# Then we find out who are the active users in our population by using this expresion. 
# Note:
# - the '&' signifies a lofgical AND
# - the '|' signifies a lofgical OR
active_redditors = df4.loc[
                           (df4['comments'] != 1000) & 
                            # note the use of brackets to isolate the OR expression
                           ((df4['unique_subs'] >= 10) | (df4['comments_per_sub'] >= 3))  
                          ]
msg = f'there are {len(active_redditors)} active redditors in our sample'
print(msg)

there are 120 active redditors in our sample


#### Exercise 2

1. Find each subreddit, find how many users have the string "trump" (case insensitive) in their name, and how many comments they have made
2. What are the top 10 subreddits who have the most comments made by users with "trump" in their name
3. Can you adapt the code to look for any string
For each user having the substring "trump" in its name

In [52]:
## exercise code goes here



##

username
presidenttrump_2016    1000
TrumpsMurica           1000
PumpTrump              1000
Name: utc, dtype: int64

#### Exercise 3

1. Find the top 10 subreddits with the most users
2. For active redditors, find the median number of comments, the 25th and 75th percentile of unique_subs
3. Find the subreddit with the most comments in one hour
4. Plot a timeserie of:
    - Number of comments per day
    - Number of comments per day, only in subreddit where at least one "trump user" has commented, only comments written since 01 Jan 2016
5. [optional] Recompute the active redditors in a single expression    



In [66]:
## exercise code goes here






##

# Solutions

## Exercise 1

1. Extract the hour of day (ie: 10 for a datetime of 2016-12-26 10:24:59) and assign it to a new column ("hour_of_day")
2. Find the top 10 subreddits with the most users
3. Find which hour of the day is one average, generating more comments

In [110]:
# 1.1: extract the hour of day
def get_hour_of_day(elem):
    return elem.hour
                    
df['hour_of_day'] = df['utc'].apply(get_hour_of_day)
df.head()

,username,subreddit,utc,date,week_of_year,trump_user,hour,hour_of_day
0,kabanossi,photoshopbattles,2016-12-26 10:24:59,2016-12-26,52,False,10,10
1,kabanossi,GetMotivated,2016-12-26 10:23:14,2016-12-26,52,False,10,10
2,kabanossi,vmware,2016-12-26 10:21:16,2016-12-26,52,False,10,10
3,kabanossi,carporn,2016-12-26 10:20:18,2016-12-26,52,False,10,10
4,kabanossi,DIY,2016-12-26 10:17:59,2016-12-26,52,False,10,10


In [109]:
# 1.2: find the top 10 subreddits with the most users
df.groupby('subreddit').agg({'username': 'nunique'}).sort_values(by='username', ascending=False).head(10)

,username
subreddit,
AskReddit,141
news,127
pics,124
funny,122
todayilearned,122
worldnews,116
videos,108
gifs,105
GetMotivated,100


In [119]:
# 3. Find which hour of the day is one average, generating more comments
df.groupby('hour_of_day').agg({'subreddit': 'count'}).sort_values(by='subreddit', ascending=False).head()

,subreddit
hour_of_day,
15,5855
14,5707
16,5684
17,5505
18,4994


## Exercise 2

### Solution 2.A: using known syntax

In [91]:
def get_token(elem, token):
    return token.lower() in elem.lower()

# Create a dataframe containing only users with "trump" in their name
df['trump_user'] = df['username'].apply(get_token, args=['trump'])
df_trump = df.loc[df['trump_user'] == True]
df_trump.head()

username                
                    nb_comments_by_trump nb_user_w_trump
subreddit                                               
2meirl42meirl4meirl                    1               1
4chan                                 11               1
AdviceAnimals                         16               1
AmISexy                                1               1
Art                                    5               1

In [96]:
# creating the groupbys
trump_agg = (df_trump.groupby('subreddit')
         .agg({'username': [
             ('nb_comments_by_trump', 'count'),
             ('nb_user_w_trump', 'nunique')
         ]})
)
trump_agg.head()

username                
                    nb_comments_by_trump nb_user_w_trump
subreddit                                               
2meirl42meirl4meirl                    1               1
4chan                                 11               1
AdviceAnimals                         16               1
AmISexy                                1               1
Art                                    5               1

In [95]:
# cleaning and sorting
final = trump_agg.droplevel(axis='columns', level=0)
final.sort_values(by=f'nb_comments_by_trump', ascending=False).head(10)

,nb_comments_by_trump,nb_user_w_trump
subreddit,,
The_Donald,935,2
politics,750,3
PoliticalDiscussion,317,2
news,174,3
worldnews,91,3
technology,73,1
environment,66,2
videos,56,2
houston,28,1


### Solution 2.B : a cleaner way

In [98]:
token = 'trump'
mask = df['username'].str.contains(token, case=False)
(df.loc[mask]
    .groupby('subreddit')
    .agg({'username': [
            (f'nb_comments_by_{token}', 'count'),
            (f'nb_users_w_{token}', 'nunique'),
    ]
         })
     .droplevel(axis='columns', level=0)
    .sort_values(by=f'nb_comments_by_{token}', ascending=False)
    .head(10)
)

,nb_comments_by_trump,nb_users_w_trump
subreddit,,
The_Donald,935,2
politics,750,3
PoliticalDiscussion,317,2
news,174,3
worldnews,91,3
technology,73,1
environment,66,2
videos,56,2
houston,28,1


## Exercise 3

1. For active redditors, find the median number of comments, the 25th and 75th percentile of unique_subs
2. Find the subreddit with the most comments in one hour
3. [harder but useful] Recompute the active redditors in a single expression


In [127]:
# 1. For active redditors, find the median number of comments, the 25th and 75th percentile of unique_subs
median_comments = active_redditors['comments'].median()
p25_subs = np.percentile(active_redditors['unique_subs'], 25)
p75_subs = np.percentile(active_redditors['unique_subs'], 75)

msg = f"""Here are the stats on active reddits:
A. Comments per subredit:
- mediam: {median_comments}
B. Number of subreddits they subscribe to:
- p25: {p25_comments}
- p75: {p75_comments}
"""

print(msg)

Here are the stats on active reddits:
A. Comments per subredit:
- mediam: 358.0
B. Number of subreddits they subscribe to:
- p25: 26.0
- p75: 64.75



In [135]:
# 2. Find the subreddit with the most comments in one hour
def per_hour(elem: dt.datetime):
    return f'{elem.date()}_{elem.hour}'

df['per_hour'] = df['utc'].apply(per_hour)
df.groupby(['per_hour', 'subreddit'])['username'].count().sort_values(ascending=False).head()

per_hour       subreddit
2016-12-26_15  news         108
2016-04-16_20  nba           59
2016-04-19_3   nba           47
2016-03-20_1   linux         41
2016-11-30_1   FlashTV       40
Name: username, dtype: int64

In [136]:
# 3. [harder but useful] Recompute the active redditors in a single expression

# We use "method chaining" to avoid intermediate dataframes
# Advantages:
# 1. Avoids mistakes (easy to make errors when you reach df42)
# 2. You can read all the logic step by step

active_redditors = \
(df.groupby('username')
   .agg({'subreddit': [('comments', 'count'),
                       ('unique_subs', lambda x: len(set(x))),
                      ]
        })
    .droplevel(axis='columns', level=0)
    .assign(comments_per_sub=lambda x: x['comments'] / x['unique_subs'])
    .sort_values(by='comments_per_sub', ascending=False)
    .query('comments < 1000')
    .query('unique_subs >= 10')
    .query('comments_per_sub >= 3') 
)
active_redditors.head()

,comments,unique_subs,comments_per_sub
username,,,
mr_fn_wonderful,999,21,47.571429
The_seph_i_am,986,32,30.812500
laserbong,804,29,27.724138
BrandEnlightened,426,20,21.300000
QuatreBarton,336,16,21.000000
